In [4]:
import pandas as pd
import numpy as np # linear algebra

# Replace 'path_to_dataset' with the actual path to your dataset file
df = pd.read_excel('LastDataset1(neww).xlsx')
df.shape

(2088, 95)

In [5]:
X = df[['Faculty_domain', 'Employed_Company_Name', 'Industry_sector', 'Employed_Company_district', 'Employed_Company_state', 'Employed_Company_Address', 'Main_job_scope',
 'Main_job_Title', 'Work_in_the_same_field_of_learning', 'Faculty_Description', 'JS_Job_sub_sector', 'Employed_Company_lat', 'Faculty', 'Program_Description', 'Employed_Company_Postcode', 'Economic_sector', 'Current_Job_scope']]
y = df['JS_Job_sector']

categorical_columns = X.select_dtypes(include=['object']).columns
numerical_columns = X.select_dtypes(include=['int', 'float']).columns

# Create DataFrames for categorical and numerical columns
X_categorical = X[categorical_columns]
X_numerical = X[numerical_columns]

from sklearn.preprocessing import LabelEncoder



# Perform label encoding only on columns with dtype object
label_encoder = LabelEncoder()
X_encoded = X_categorical.copy()
for col in categorical_columns:
    X_encoded[col] = X[col].astype(str)  # Ensure the column is of string dtype before label encoding
    X_encoded[col] = label_encoder.fit_transform(X_encoded[col])

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()


# Initialize LabelEncoder for the target variable y
label_encoder_y = LabelEncoder()
y_encoded = label_encoder_y.fit_transform(y)
# Merge with original X containing int and float columns
X_final = pd.concat([X_encoded, X.select_dtypes(include=['int', 'float'])], axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_encoded, test_size=0.2, random_state=42)


In [6]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE

smote = SMOTE(random_state=42)
X_train_resampled_smote, y_train_resampled_smote = smote.fit_resample(X_train, y_train)

borderline_smote = BorderlineSMOTE(random_state=42)
X_train_resampled_borderline, y_train_resampled_borderline = borderline_smote.fit_resample(X_train, y_train)


## BorutaShap SMOTE + Bordeline SMOTE
focus on Bordeline SMOTE only also can

RF

In [7]:
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Train the classifier
rf_classifier.fit(X_train_resampled_smote, y_train_resampled_smote)
# Define the parameter grid
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=rf_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_rf_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_rf = best_rf_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)
classification_report_result_best_rf = classification_report(y_test, y_pred_best_rf, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Random Forest with SMOTE): {accuracy_best_rf}")
print("Classification Report (Best Random Forest with SMOTE):\n", classification_report_result_best_rf)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters: {'max_depth': None, 'n_estimators': 200}
Accuracy (Best Random Forest with SMOTE): 0.8947368421052632
Classification Report (Best Random Forest with SMOTE):
               precision    recall  f1-score   support

           0      0.875     0.840     0.857        50
           1      0.794     0.833     0.813        60
           2      0.667     0.714     0.690        14
           3      0.500     0.500     0.500         2
           4      1.000     0.987     0.993        76
           5      0.917     0.846     0.880        13
           6      0.950     1.000     0.974        19
           7      0.889     1.000     0.941         8
           8      0.750     0.600     0.667         5
           9      1.000     1.000     1.000        94
          10      0.615     0.571     0.593        14
          11      0.697     0.793     0.742        29
          12      1.000     0.882     0.938        34

    a

In [8]:
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize Borderline SMOTE

# Define the parameter grid for GridSearchCV
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=rf_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV with resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_rf_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_rf = best_rf_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_rf = accuracy_score(y_test, y_pred_best_rf)
classification_report_result_best_rf = classification_report(y_test, y_pred_best_rf, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Random Forest with Borderline SMOTE): {accuracy_best_rf}")
print("Classification Report (Best Random Forest with Borderline SMOTE):\n", classification_report_result_best_rf)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters: {'max_depth': None, 'n_estimators': 150}
Accuracy (Best Random Forest with Borderline SMOTE): 0.8803827751196173
Classification Report (Best Random Forest with Borderline SMOTE):
               precision    recall  f1-score   support

           0      0.857     0.840     0.848        50
           1      0.814     0.800     0.807        60
           2      0.750     0.643     0.692        14
           3      0.000     0.000     0.000         2
           4      0.987     1.000     0.993        76
           5      1.000     0.846     0.917        13
           6      0.905     1.000     0.950        19
           7      0.727     1.000     0.842         8
           8      1.000     0.600     0.750         5
           9      1.000     1.000     1.000        94
          10      0.333     0.143     0.200        14
          11      0.568     0.862     0.685        29
          12      1.000     0.912     

d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Gaussian NB

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
# Define the parameter grid
param_grid = {
    'var_smoothing': [1e-9, 1.778e-7, 3.162e-5, 5.6e-3, 1]
}

# Initialize the GaussianNB classifier
gnb_classifier = GaussianNB()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=gnb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_gnb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_gnb = best_gnb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_gnb = accuracy_score(y_test, y_pred_best_gnb)
classification_report_result_best_gnb = classification_report(y_test, y_pred_best_gnb, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best GaussianNB with SMOTE): {accuracy_best_gnb}")
print("Classification Report (Best GaussianNB with SMOTE):\n", classification_report_result_best_gnb)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'var_smoothing': 1.778e-07}
Accuracy (Best GaussianNB with SMOTE): 0.7416267942583732
Classification Report (Best GaussianNB with SMOTE):
               precision    recall  f1-score   support

           0      0.720     0.720     0.720        50
           1      0.733     0.550     0.629        60
           2      0.667     0.571     0.615        14
           3      0.000     0.000     0.000         2
           4      1.000     0.816     0.899        76
           5      0.500     0.615     0.552        13
           6      0.708     0.895     0.791        19
           7      0.778     0.875     0.824         8
           8      0.250     0.400     0.308         5
           9      1.000     1.000     1.000        94
          10      0.278     0.357     0.312        14
          11      0.300     0.310     0.305        29
          12      0.674     0.853     0.753        34

    accuracy             

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
# Define the parameter grid
param_grid = {
    'var_smoothing': [1e-9, 1.778e-7, 3.162e-5, 5.6e-3, 1]
}

# Initialize the GaussianNB classifier
gnb_classifier = GaussianNB()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=gnb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_gnb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_gnb = best_gnb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_gnb = accuracy_score(y_test, y_pred_best_gnb)
classification_report_result_best_gnb = classification_report(y_test, y_pred_best_gnb, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best GaussianNB with SMOTE): {accuracy_best_gnb}")
print("Classification Report (Best GaussianNB with SMOTE):\n", classification_report_result_best_gnb)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'var_smoothing': 1e-09}
Accuracy (Best GaussianNB with SMOTE): 0.7631578947368421
Classification Report (Best GaussianNB with SMOTE):
               precision    recall  f1-score   support

           0      0.741     0.800     0.769        50
           1      0.825     0.550     0.660        60
           2      0.444     0.571     0.500        14
           3      0.000     0.000     0.000         2
           4      1.000     0.829     0.906        76
           5      0.600     0.692     0.643        13
           6      0.708     0.895     0.791        19
           7      1.000     0.625     0.769         8
           8      0.400     0.400     0.400         5
           9      1.000     1.000     1.000        94
          10      0.316     0.429     0.364        14
          11      0.371     0.448     0.406        29
          12      0.644     0.853     0.734        34

    accuracy                 

DT

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Define the parameter grid
param_grid = {
    'max_depth': [2, 3, 4, 5, None]
}

# Initialize the Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=dt_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_dt_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_dt = best_dt_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_dt = accuracy_score(y_test, y_pred_best_dt)
classification_report_result_best_dt = classification_report(y_test, y_pred_best_dt, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Decision Tree with SMOTE): {accuracy_best_dt}")
print("Classification Report (Best Decision Tree with SMOTE):\n", classification_report_result_best_dt)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'max_depth': None}
Accuracy (Best Decision Tree with SMOTE): 0.8875598086124402
Classification Report (Best Decision Tree with SMOTE):
               precision    recall  f1-score   support

           0      0.904     0.940     0.922        50
           1      0.900     0.900     0.900        60
           2      0.667     0.571     0.615        14
           3      0.250     0.500     0.333         2
           4      1.000     0.934     0.966        76
           5      0.684     1.000     0.812        13
           6      0.810     0.895     0.850        19
           7      1.000     0.375     0.545         8
           8      0.667     0.400     0.500         5
           9      1.000     1.000     1.000        94
          10      0.611     0.786     0.688        14
          11      0.655     0.655     0.655        29
          12      0.969     0.912     0.939        34

    accuracy                

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Define the parameter grid
param_grid = {
    'max_depth': [2, 3, 4, 5, None]
}

# Initialize the Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=dt_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_dt_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_dt = best_dt_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_dt = accuracy_score(y_test, y_pred_best_dt)
classification_report_result_best_dt = classification_report(y_test, y_pred_best_dt, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best Decision Tree with SMOTE): {accuracy_best_dt}")
print("Classification Report (Best Decision Tree with SMOTE):\n", classification_report_result_best_dt)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'max_depth': None}
Accuracy (Best Decision Tree with SMOTE): 0.8779904306220095
Classification Report (Best Decision Tree with SMOTE):
               precision    recall  f1-score   support

           0      0.800     0.880     0.838        50
           1      0.786     0.733     0.759        60
           2      0.636     0.500     0.560        14
           3      0.000     0.000     0.000         2
           4      1.000     1.000     1.000        76
           5      0.857     0.923     0.889        13
           6      0.905     1.000     0.950        19
           7      0.875     0.875     0.875         8
           8      0.571     0.800     0.667         5
           9      1.000     1.000     1.000        94
          10      0.562     0.643     0.600        14
          11      0.769     0.690     0.727        29
          12      0.912     0.912     0.912        34

    accuracy                

d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\python\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


KNN


In [13]:
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assume X_final and y_encoded are your features and target after preprocessing


# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE to the training data only
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_neighbors': [1, 2, 3, 4, 5]
}

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=knn_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV with resampled data
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_knn_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_knn = best_knn_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_knn = accuracy_score(y_test, y_pred_best_knn)
classification_report_result_best_knn = classification_report(y_test, y_pred_best_knn, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best KNN with SMOTE): {accuracy_best_knn}")
print("Classification Report (Best KNN with SMOTE):\n", classification_report_result_best_knn)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'n_neighbors': 1}
Accuracy (Best KNN with SMOTE): 0.6770334928229665
Classification Report (Best KNN with SMOTE):
               precision    recall  f1-score   support

           0      0.667     0.600     0.632        50
           1      0.657     0.383     0.484        60
           2      0.389     0.500     0.438        14
           3      0.000     0.000     0.000         2
           4      0.753     0.803     0.777        76
           5      0.700     0.538     0.609        13
           6      0.560     0.737     0.636        19
           7      0.500     0.625     0.556         8
           8      0.250     0.200     0.222         5
           9      0.979     1.000     0.989        94
          10      0.214     0.214     0.214        14
          11      0.387     0.414     0.400        29
          12      0.591     0.765     0.667        34

    accuracy                          0.677      

In [14]:
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Assume X_final and y_encoded are your features and target after preprocessing


# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply Borderline SMOTE to the training data only
smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_neighbors': [1, 2, 3, 4, 5]
}

# Initialize the KNN classifier
knn_classifier = KNeighborsClassifier()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=knn_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV with resampled data
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_knn_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_knn = best_knn_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_knn = accuracy_score(y_test, y_pred_best_knn)
classification_report_result_best_knn = classification_report(y_test, y_pred_best_knn, digits=3)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy (Best KNN with Borderline SMOTE): {accuracy_best_knn}")
print("Classification Report (Best KNN with Borderline SMOTE):\n", classification_report_result_best_knn)


Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best Parameters: {'n_neighbors': 1}
Accuracy (Best KNN with Borderline SMOTE): 0.6889952153110048
Classification Report (Best KNN with Borderline SMOTE):
               precision    recall  f1-score   support

           0      0.640     0.640     0.640        50
           1      0.639     0.383     0.479        60
           2      0.474     0.643     0.545        14
           3      0.000     0.000     0.000         2
           4      0.772     0.803     0.787        76
           5      0.778     0.538     0.636        13
           6      0.636     0.737     0.683        19
           7      0.625     0.625     0.625         8
           8      0.167     0.200     0.182         5
           9      0.979     1.000     0.989        94
          10      0.300     0.214     0.250        14
          11      0.389     0.483     0.431        29
          12      0.556     0.735     0.633        34

    accuracy               

SVM

In [15]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE



# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for SVM
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
}

# Initialize the SVM classifier
svm_classifier = SVC()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=svm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_svm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_svm = best_svm_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_svm = accuracy_score(y_test, y_pred_best_svm)
classification_report_result_best_svm = classification_report(y_test, y_pred_best_svm, digits=3)

# Print results
print(f"Best Parameters (SVM with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (SVM with SMOTE): {accuracy_best_svm}")
print("Classification Report (SVM with SMOTE):\n", classification_report_result_best_svm)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Parameters (SVM with SMOTE): {'C': 100, 'gamma': 0.1}
Accuracy (SVM with SMOTE): 0.7464114832535885
Classification Report (SVM with SMOTE):
               precision    recall  f1-score   support

           0      0.722     0.780     0.750        50
           1      0.660     0.550     0.600        60
           2      0.438     0.500     0.467        14
           3      0.000     0.000     0.000         2
           4      0.807     0.882     0.843        76
           5      0.889     0.615     0.727        13
           6      0.789     0.789     0.789        19
           7      0.571     0.500     0.533         8
           8      0.333     0.200     0.250         5
           9      0.989     1.000     0.995        94
          10      0.250     0.071     0.111        14
          11      0.436     0.586     0.500        29
          12      0.703     0.765     0.732        34

    accuracy                       

In [16]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import BorderlineSMOTE

# Assuming X_train, X_test, y_train, y_test are already defined
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply BorderlineSMOTE
borderline_smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = borderline_smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for SVM
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
}

# Initialize the SVM classifier
svm_classifier = SVC()

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=svm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_svm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_svm = best_svm_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_svm = accuracy_score(y_test, y_pred_best_svm)
classification_report_result_best_svm = classification_report(y_test, y_pred_best_svm, digits=3)

# Print results
print(f"Best Parameters (SVM with BorderlineSMOTE): {grid_search.best_params_}")
print(f"Accuracy (SVM with BorderlineSMOTE): {accuracy_best_svm}")
print("Classification Report (SVM with BorderlineSMOTE):\n", classification_report_result_best_svm)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Parameters (SVM with BorderlineSMOTE): {'C': 10, 'gamma': 0.1}
Accuracy (SVM with BorderlineSMOTE): 0.7488038277511961
Classification Report (SVM with BorderlineSMOTE):
               precision    recall  f1-score   support

           0      0.765     0.780     0.772        50
           1      0.680     0.567     0.618        60
           2      0.412     0.500     0.452        14
           3      0.000     0.000     0.000         2
           4      0.835     0.868     0.852        76
           5      0.800     0.615     0.696        13
           6      0.727     0.842     0.780        19
           7      0.667     0.500     0.571         8
           8      0.500     0.200     0.286         5
           9      0.989     1.000     0.995        94
          10      0.250     0.071     0.111        14
          11      0.405     0.586     0.479        29
          12      0.684     0.765     0.722        34

    ac

LR

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Assuming X_train, X_test, y_train, y_test are already defined
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for Logistic Regression
param_grid = {
    'C': [0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear'],  # Choose either 'lbfgs' or 'liblinear' solver
}

# Initialize the Logistic Regression classifier
lr_classifier = LogisticRegression(max_iter=1000)  # Increase max_iter if necessary

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lr_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_lr_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lr = best_lr_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_lr = accuracy_score(y_test, y_pred_best_lr)
classification_report_result_best_lr = classification_report(y_test, y_pred_best_lr, digits=3)

# Print results
print(f"Best Parameters (Logistic Regression with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (Logistic Regression with SMOTE): {accuracy_best_lr}")
print("Classification Report (Logistic Regression with SMOTE):\n", classification_report_result_best_lr)


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Parameters (Logistic Regression with SMOTE): {'C': 100, 'solver': 'lbfgs'}
Accuracy (Logistic Regression with SMOTE): 0.6626794258373205
Classification Report (Logistic Regression with SMOTE):
               precision    recall  f1-score   support

           0      0.767     0.660     0.710        50
           1      0.743     0.433     0.547        60
           2      0.583     0.500     0.538        14
           3      0.000     0.000     0.000         2
           4      0.906     0.632     0.744        76
           5      0.500     0.538     0.519        13
           6      0.548     0.895     0.680        19
           7      0.261     0.750     0.387         8
           8      0.154     0.400     0.222         5
           9      0.969     1.000     0.984        94
          10      0.278     0.357     0.312        14
          11      0.231     0.207     0.218        29
          12      0.684     0.765     

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import BorderlineSMOTE

# Assuming X_train, X_test, y_train, y_test are already defined
# Apply StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply BorderlineSMOTE
smote = BorderlineSMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define the parameter grid for Logistic Regression
param_grid = {
    'C': [0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear'],  # Choose either 'lbfgs' or 'liblinear' solver
}

# Initialize the Logistic Regression classifier
lr_classifier = LogisticRegression(max_iter=1000)  # Increase max_iter if necessary

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lr_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best model
best_lr_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lr = best_lr_classifier.predict(X_test_scaled)

# Evaluate the best model
accuracy_best_lr = accuracy_score(y_test, y_pred_best_lr)
classification_report_result_best_lr = classification_report(y_test, y_pred_best_lr, digits=3)

# Print results
print(f"Best Parameters (Logistic Regression with BorderlineSMOTE): {grid_search.best_params_}")
print(f"Accuracy (Logistic Regression with BorderlineSMOTE): {accuracy_best_lr}")
print("Classification Report (Logistic Regression with BorderlineSMOTE):\n", classification_report_result_best_lr)


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Parameters (Logistic Regression with BorderlineSMOTE): {'C': 100, 'solver': 'lbfgs'}
Accuracy (Logistic Regression with BorderlineSMOTE): 0.6555023923444976
Classification Report (Logistic Regression with BorderlineSMOTE):
               precision    recall  f1-score   support

           0      0.739     0.680     0.708        50
           1      0.692     0.450     0.545        60
           2      0.636     0.500     0.560        14
           3      0.000     0.000     0.000         2
           4      0.849     0.592     0.698        76
           5      0.500     0.462     0.480        13
           6      0.500     0.842     0.627        19
           7      0.278     0.625     0.385         8
           8      0.067     0.200     0.100         5
           9      0.959     1.000     0.979        94
          10      0.263     0.357     0.303        14
          11      0.308     0.276     0.291        29
        

XGBoost

In [19]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for XGBoost
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, None],
}

# Initialize the XGBoost classifier
xgb_classifier = XGBClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_xgb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_xgb = best_xgb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_xgb = accuracy_score(y_test, y_pred_best_xgb)
classification_report_result_best_xgb = classification_report(y_test, y_pred_best_xgb, digits=3)

# Print results
print(f"Best Parameters (XGBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (XGBoost with SMOTE): {accuracy_best_xgb}")
print("Classification Report (XGBoost with SMOTE):\n", classification_report_result_best_xgb)


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Parameters (XGBoost with SMOTE): {'gamma': None, 'max_depth': 3}
Accuracy (XGBoost with SMOTE): 0.937799043062201
Classification Report (XGBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.960     0.960     0.960        50
           1      0.917     0.917     0.917        60
           2      0.769     0.714     0.741        14
           3      0.000     0.000     0.000         2
           4      1.000     0.987     0.993        76
           5      0.857     0.923     0.889        13
           6      0.950     1.000     0.974        19
           7      0.889     1.000     0.941         8
           8      1.000     0.800     0.889         5
           9      1.000     1.000     1.000        94
          10      0.818     0.643     0.720        14
          11      0.788     0.897     0.839        29
          12      0.970     0.941     0.955        34

    accuracy    

In [20]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for XGBoost
param_grid = {
    'max_depth': [2, 3, 4, 5, None],
    'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, None],
}

# Initialize the XGBoost classifier
xgb_classifier = XGBClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_xgb_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_xgb = best_xgb_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_xgb = accuracy_score(y_test, y_pred_best_xgb)
classification_report_result_best_xgb = classification_report(y_test, y_pred_best_xgb, digits=3)

# Print results
print(f"Best Parameters (XGBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (XGBoost with SMOTE): {accuracy_best_xgb}")
print("Classification Report (XGBoost with SMOTE):\n", classification_report_result_best_xgb)


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Parameters (XGBoost with SMOTE): {'gamma': None, 'max_depth': 4}
Accuracy (XGBoost with SMOTE): 0.9425837320574163
Classification Report (XGBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.959     0.940     0.949        50
           1      0.900     0.900     0.900        60
           2      0.769     0.714     0.741        14
           3      1.000     0.500     0.667         2
           4      1.000     1.000     1.000        76
           5      0.923     0.923     0.923        13
           6      0.905     1.000     0.950        19
           7      0.889     1.000     0.941         8
           8      1.000     1.000     1.000         5
           9      1.000     1.000     1.000        94
          10      0.786     0.786     0.786        14
          11      0.862     0.862     0.862        29
          12      0.941     0.941     0.941        34

    accuracy   

In [21]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the XGBoost classifier with default parameters
xgb_classifier_baseline = XGBClassifier(random_state=42)

# Fit the classifier on the training data (assuming X_train and y_train are defined)
xgb_classifier_baseline.fit(X_train, y_train)

# Make predictions on the test set
y_pred_xgb_baseline = xgb_classifier_baseline.predict(X_test)

# Evaluate the baseline model
accuracy_xgb_baseline = accuracy_score(y_test, y_pred_xgb_baseline)
classification_report_result_xgb_baseline = classification_report(y_test, y_pred_xgb_baseline, digits=3)

# Print results
print("Baseline XGBoost Model:")
print(f"Accuracy: {accuracy_xgb_baseline}")
print("Classification Report:\n", classification_report_result_xgb_baseline)


Baseline XGBoost Model:
Accuracy: 0.9880382775119617
Classification Report:
               precision    recall  f1-score   support

           0      1.000     0.960     0.980        50
           1      0.968     1.000     0.984        60
           2      1.000     0.929     0.963        14
           3      1.000     0.500     0.667         2
           4      1.000     1.000     1.000        76
           5      1.000     0.923     0.960        13
           6      0.950     1.000     0.974        19
           7      1.000     1.000     1.000         8
           8      1.000     1.000     1.000         5
           9      1.000     1.000     1.000        94
          10      0.875     1.000     0.933        14
          11      1.000     1.000     1.000        29
          12      1.000     1.000     1.000        34

    accuracy                          0.988       418
   macro avg      0.984     0.947     0.959       418
weighted avg      0.989     0.988     0.988       418



AdaBoost

In [22]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity


# Define the parameter grid for AdaBoost
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.001, 0.1, 1],
}

# Initialize the AdaBoost classifier
ada_classifier = AdaBoostClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=ada_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV on resampled data
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_ada_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_ada = best_ada_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_ada = accuracy_score(y_test, y_pred_best_ada)
classification_report_result_best_ada = classification_report(y_test, y_pred_best_ada, digits=3)

# Print results
print(f"Best Parameters (AdaBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (AdaBoost with SMOTE): {accuracy_best_ada}")
print("Classification Report (AdaBoost with SMOTE):\n", classification_report_result_best_ada)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters (AdaBoost with SMOTE): {'learning_rate': 0.1, 'n_estimators': 50}
Accuracy (AdaBoost with SMOTE): 0.4784688995215311
Classification Report (AdaBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.857     0.120     0.211        50
           1      0.207     0.583     0.306        60
           2      0.314     0.786     0.449        14
           3      0.118     1.000     0.211         2
           4      1.000     1.000     1.000        76
           5      0.786     0.846     0.815        13
           6      0.842     0.842     0.842        19
           7      0.000     0.000     0.000         8
           8      0.000     0.000     0.000         5
           9      0.000     0.000     0.000        94
          10      0.222     0.143     0.174        14
          11      0.286     0.414     0.338        29
          12      1.000     0.853     0.921        34

  

In [23]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity


# Define the parameter grid for AdaBoost
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.001, 0.1, 1],
}

# Initialize the AdaBoost classifier
ada_classifier = AdaBoostClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=ada_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV on resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_ada_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_ada = best_ada_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_ada = accuracy_score(y_test, y_pred_best_ada)
classification_report_result_best_ada = classification_report(y_test, y_pred_best_ada, digits=3)

# Print results
print(f"Best Parameters (AdaBoost with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (AdaBoost with SMOTE): {accuracy_best_ada}")
print("Classification Report (AdaBoost with SMOTE):\n", classification_report_result_best_ada)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters (AdaBoost with SMOTE): {'learning_rate': 0.1, 'n_estimators': 50}
Accuracy (AdaBoost with SMOTE): 0.5454545454545454
Classification Report (AdaBoost with SMOTE):
               precision    recall  f1-score   support

           0      0.656     0.840     0.737        50
           1      0.197     0.417     0.267        60
           2      0.297     0.786     0.431        14
           3      0.000     0.000     0.000         2
           4      1.000     1.000     1.000        76
           5      1.000     0.846     0.917        13
           6      0.800     0.632     0.706        19
           7      0.000     0.000     0.000         8
           8      0.000     0.000     0.000         5
           9      0.000     0.000     0.000        94
          10      0.091     0.071     0.080        14
          11      0.541     0.690     0.606        29
          12      1.000     0.882     0.938        34

  

In [24]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the AdaBoost classifier with default parameters
ada_classifier_baseline = AdaBoostClassifier(random_state=42)

# Fit the classifier on the training data (assuming X_train and y_train are defined)
ada_classifier_baseline.fit(X_train, y_train)

# Make predictions on the test set
y_pred_ada_baseline = ada_classifier_baseline.predict(X_test)

# Evaluate the baseline model
accuracy_ada_baseline = accuracy_score(y_test, y_pred_ada_baseline)
classification_report_result_ada_baseline = classification_report(y_test, y_pred_ada_baseline, digits=3)

# Print results
print("Baseline AdaBoost Model:")
print(f"Accuracy: {accuracy_ada_baseline}")
print("Classification Report:\n", classification_report_result_ada_baseline)


Baseline AdaBoost Model:
Accuracy: 0.69377990430622
Classification Report:
               precision    recall  f1-score   support

           0      0.000     0.000     0.000        50
           1      0.362     0.900     0.517        60
           2      0.000     0.000     0.000        14
           3      0.000     0.000     0.000         2
           4      1.000     1.000     1.000        76
           5      1.000     0.846     0.917        13
           6      0.000     0.000     0.000        19
           7      0.000     0.000     0.000         8
           8      0.000     0.000     0.000         5
           9      1.000     1.000     1.000        94
          10      0.000     0.000     0.000        14
          11      0.439     0.862     0.581        29
          12      1.000     0.882     0.938        34

    accuracy                          0.694       418
   macro avg      0.369     0.422     0.381       418
weighted avg      0.602     0.694     0.626       418



CatBoost

LGBM

In [25]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for LGBM
param_grid = {
    'max_depth': [-1, 2, 3, 4, 5],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the LGBM classifier
lgbm_classifier = LGBMClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lgbm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Assuming X_train_resampled_smote, y_train_resampled_smote are defined
# Fit GridSearchCV on SMOTE resampled data
grid_search.fit(X_train_resampled_smote, y_train_resampled_smote)

# Get the best model
best_lgbm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lgbm = best_lgbm_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_lgbm = accuracy_score(y_test, y_pred_best_lgbm)
classification_report_result_best_lgbm = classification_report(y_test, y_pred_best_lgbm, digits=3)

# Print results
print(f"Best Parameters (LGBM with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (LGBM with SMOTE): {accuracy_best_lgbm}")
print("Classification Report (LGBM with SMOTE):\n", classification_report_result_best_lgbm)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000258 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1513
[LightGBM] [Info] Number of data points in the train set: 4706, number of used features: 17
[LightGBM] [Info] Start training from score -2.564949
[LightGBM] [Info] Start training from score -2.564949
[LightGBM] [Info] Start training from score -2.564949
[LightGBM] [Info] Start training from score -2.564949
[LightGBM] [Info] Start training from score -2.564949
[LightGBM] [Info] Start training from score -2.564949
[LightGBM] [Info] Start training from score -2.564949
[LightGBM] [

In [26]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')  # Ignore warnings for simplicity

# Define the parameter grid for LGBM
param_grid = {
    'max_depth': [-1, 2, 3, 4, 5],
    'n_estimators': [50, 100, 150, 200],
}

# Initialize the LGBM classifier
lgbm_classifier = LGBMClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=lgbm_classifier,
    param_grid=param_grid,
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1  # Use all available cores
)

# Assuming X_train_resampled_smote, y_train_resampled_smote are defined
# Fit GridSearchCV on SMOTE resampled data
grid_search.fit(X_train_resampled_borderline, y_train_resampled_borderline)

# Get the best model
best_lgbm_classifier = grid_search.best_estimator_

# Make predictions on the test set using the best model
y_pred_best_lgbm = best_lgbm_classifier.predict(X_test)

# Evaluate the best model
accuracy_best_lgbm = accuracy_score(y_test, y_pred_best_lgbm)
classification_report_result_best_lgbm = classification_report(y_test, y_pred_best_lgbm, digits=3)

# Print results
print(f"Best Parameters (LGBM with SMOTE): {grid_search.best_params_}")
print(f"Accuracy (LGBM with SMOTE): {accuracy_best_lgbm}")
print("Classification Report (LGBM with SMOTE):\n", classification_report_result_best_lgbm)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000210 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1502
[LightGBM] [Info] Number of data points in the train set: 4352, number of used features: 17
[LightGBM] [Info] Start training from score -2.486747
[LightGBM] [Info] Start training from score -2.486747
[LightGBM] [Info] Start training from score -2.486747
[LightGBM] [Info] Start training from score -6.298949
[LightGBM] [Info] Start training from score -2.486747
[LightGBM] [Info] Start training from score -2.486747
[LightGBM] [Info] Start training from score -2.486747
[LightGBM] [